In [1]:
import sys
sys.path.append("/home/juliafalcao")
sys.path.append("/home/juliafalcao/experiments")

from experiments.constants import * 
from experiments.utils import *

import pandas as pd
import json

In [2]:
SRC, TGT = SPANISH, BASQUE

In [3]:
# batches_filepath = f"/netscratch/falcao/data/{SRC}-{TGT}/eval-set/batches.{SRC}-{TGT}.json"
batches_filepath = f"/home/juliafalcao/thesis_data/{SRC}-{TGT}/batches.{SRC}-{TGT}.json"
with open(batches_filepath, mode="r") as f:
    j = json.load(f)

In [17]:
ref_batches = []

for i, batch in enumerate(j):
    ref_batch = {
        "items": [item for item in batch["items"] if item["itemType"] == "REF"],
        "task": {
            "batchNo": i+1,
            "batchSize": 10,
            "randomSeed": 123456,
            "requiredAnnotations": 3,
            "sourceLanguage": "spa",
            "targetLanguage": "eus"
        }
    }

    ref_batches.append(ref_batch)

In [27]:
with open(f"/home/juliafalcao/thesis_data/{SRC}-{TGT}/ref-batches.{SRC}-{TGT}.json", mode="w+") as f:
    json.dump(ref_batches, f, indent=4)

In [4]:
df_filepath = f"/home/juliafalcao/thesis_data/{SRC}-{TGT}/df_eval.csv"
df_eval = pd.read_csv(df_filepath)

In [5]:
dfs = []
for batch_json in j:
    _df = pd.DataFrame(batch_json["items"])
    _df["_batchNo"] = batch_json["task"]["batchNo"]
    dfs.append(_df)

df = pd.concat(dfs)
df = df[["_batchNo", *df.columns[:-1]]]

In [6]:
print("total items:", len(df))

total items: 1500


In [7]:
len(df)

1500

In [9]:
df[(df.itemType == "REF")]

,_batchNo,_block,_item,itemID,itemType,sourceID,sourceText,targetID,targetText
20,1,2,20,12,REF,src.es,Un autor debe ser real en el tratamiento de su...,ref.eu,Egileak egiazkotasuna bilatu behar du bere gai...
24,1,2,24,233,REF,src.es,Mientras otras especies están confinadas a lug...,ref.eu,Orain beste espezie batzuk beren geneek egokit...
38,1,3,38,78,REF,src.es,Los artículos demandaban la aprobación unánime...,ref.eu,Estatu guztien onespen bateratua behar zuten a...
44,1,4,44,334,REF,src.es,En los años ochenta un gran número de empresas...,ref.eu,80 hamarkadan enpresa lehiakide asko merkatuan...
46,1,4,46,53,REF,src.es,Yo... yo no las puedo comprar porque mi novia ...,ref.eu,Nik... ezin ditut erosi nire neska begetariano...
...,...,...,...,...,...,...,...,...,...
60,15,146,60,116,REF,src.es,El problema radica en que la nieve es pegajosa...,ref.eu,"Elurra itsaskorra delako sortzen da arazoa, er..."
61,15,146,61,214,REF,src.es,Viajar en avión puede generar miedo en persona...,ref.eu,Hegazkinez bidaiatzea beldurgarria suerta daki...
74,15,147,74,399,REF,src.es,Cruza la línea de lo real y lo irreal mientras...,ref.eu,Jokoa aurrera doan eeinean errealitatea eta ir...
84,15,148,84,292,REF,src.es,Algo muy interesante está pasando aquí en la c...,ref.eu,Zerbait oso interesgarria gertatzen da zirkunb...


In [25]:
len(df[df.itemType != "TGT"])

300

In [16]:
df.sample(5, random_state=1)

,_batchNo,_block,_item,itemID,itemType,sourceID,sourceText,targetID,targetText
91,1,9,91,292,TGT,src.en,She always took care of the yard work herself....,gtrans.mt,Dejjem kienet tieħu ħsieb ix-xogħol tal-bitħa ...
75,1,7,75,323,TGT,src.en,He already knows the language quite well but h...,kurt.mt,huwa diġà jaf il-lingwa b'mod pjuttost tajjeb ...
64,13,126,64,87,TGT,src.en,"Over multiple centuries, the Roman empire led ...",nllb.mt,"Matul bosta sekli, l- Imperu Rumanu wassal għa..."
30,4,33,30,94,REF,src.en,Proper nutritional practices alone cannot gene...,ref.mt,Prattiċi nutrittivi korretti waħedhom ma jistg...
49,14,134,49,106,TGT,src.en,"Its frequent portrayal in music, films, litera...",nllb.mt,"Ir-ritratt frekwenti tagħha fil-mużika, fil-fi..."


In [45]:
identical_hyps = df[df.targetID.apply(lambda id: "+" in id)]
print("identical hypotheses:", len(identical_hyps))
identical_hyps

identical hypotheses: 0


,_batchNo,_block,_item,itemID,itemType,sourceID,sourceText,targetID,targetText


In [37]:
print("sourceID values:", set(df.sourceID))
assert len(set(df.sourceID)) == 1 # ? not sure

sourceID values: {'src.es'}


In [38]:
print("targetID values:", df.targetID.unique().tolist())

targetID values: ['cmbacktrans.eu', 'euskadi.eu', 'nllb.eu', 'ref.eu', 'cmbacktrans.eu+euskadi.eu']


In [39]:
print("itemType values:", set(df.itemType))
assert set(df.itemType) <= set(["BAD", "CHK", "REF", "TGT"])

print("value counts:")
print(df.itemType.value_counts().to_string())
print()

print("per batch:")
_df = df[df._batchNo == 1]
print(_df.itemType.value_counts().to_string())

itemType values: {'TGT', 'REF', 'BAD'}
value counts:
TGT    1200
BAD     150
REF     150

per batch:
TGT    80
BAD    10
REF    10
